In [1]:
import numpy as np
import pandas as pd
from collections import defaultdict
import datatable as dt
import lightgbm as lgb
from matplotlib import pyplot as plt
import gc
np.seterr(divide='ignore',invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [2]:
data_types_dict ={
    'user_id': 'int32', 
    'timestamp': 'int64',
    'content_id': 'int16', 
    'answered_correctly': 'int8', 
    'prior_question_elapsed_time': 'float32', 
    'prior_question_had_explanation': 'bool'
}
target = 'answered_correctly'

In [5]:
dir_path = '/home/zuoyuhui/datasets/riid准确回答/'
file_train = 'train.csv'
file_questions = 'questions.csv'
file_lectures = 'lectures.csv'

In [6]:
debug = True

In [7]:
train_df = dt.fread(dir_path+file_train,columns = set(data_types_dict.keys())).to_pandas()

In [8]:
print('Training dataset detailed information')
print('*' * 50)
print('Columns:', train_df.columns)
print('*' * 50)
print('Shape:', train_df.shape)
print('*' * 50)
print('NA values in each column:', sum(train_df.isna().sum()))
print('*' * 50)

Training dataset detailed information
**************************************************
Columns: Index(['timestamp', 'user_id', 'content_id', 'answered_correctly',
       'prior_question_elapsed_time', 'prior_question_had_explanation'],
      dtype='object')
**************************************************
Shape: (101230332, 6)
**************************************************
NA values in each column: 2744044
**************************************************


In [9]:
# 获取target 非 -1的样本
train_df = train_df[train_df[target]!=-1].reset_index(drop=True,inplace=False)
# 用False填充nan
train_df['prior_question_had_explanation'].fillna(False,inplace=True)
train_df = train_df.astype(data_types_dict)

In [10]:
train_df['lag'] = train_df.groupby('user_id')[target].shift()
# 列方向上求累积和 和累积个数
cum = train_df.groupby('user_id')['lag'].agg(['cumsum','cumcount'])
# 学习进步的增长率
train_df['user_correctness'] = cum['cumsum']/cum['cumcount']
train_df.drop(columns=['lag'],inplace=True)

In [11]:
def clear_mem():
    %reset -f out
    %reset -f in
    gc.collect()
    
clear_mem()

Flushing output cache (1 entries)
Flushing input history


In [12]:
# overall correctness of users 用户回答问题正确的比例，数目和次数
# sum是回答正确的次数，count是回答的题目的总次数
user_agg = train_df.groupby('user_id')[target].agg(['sum','count'])
# overall difficulty of questions 每个content出现的次数和被回答正确的比例
content_agg = train_df.groupby('content_id')[target].agg(['sum','count'])

In [25]:
# 只对每位用户进行10次最后观察
train_df = train_df.groupby('user_id').tail(10).reset_index(drop=True)

In [26]:
questions_df = pd.read_csv(
    dir_path + file_questions, 
    usecols = [0, 3],
    dtype = {'question_id': 'int16', 'part': 'int8'}
)

In [30]:
user_agg

,sum,count
user_id,,
115,32.0,46
124,7.0,30
2746,11.0,19
5382,84.0,125
8623,70.0,109
...,...,...
2147470770,173.0,226
2147470777,520.0,752
2147481750,38.0,50


In [28]:
train_df = pd.merge(train_df, questions_df, left_on = 'content_id', right_on = 'question_id', how = 'left')
train_df.drop(columns = ['question_id'], inplace = True)

In [29]:
# 某讲座被回答的次数
train_df['content_count'] = train_df['content_id'].map(content_agg['count']).astype('int32')

#某讲座被回答的正确比例
train_df['content_id'] = train_df['content_id'].map(content_agg('sum')/content_agg['count'])

TypeError: 'DataFrame' object is not callable